# **TR_2021/18 - Technical report: Case crossover for cardiovascular hospitalizations and extreme events**


|Technical Report ID  |2021/18|
|--|--|
| Title |Case crossover for cardiovascular hospitalizations and extreme events|
| Authors | Júlia De Lázari, Paula Dornhofer|
| Creation Date| 2021-08|


## Databases descriptions

**inputs:** 

- hospitalizações_circulatorio.csv: Dataframe of hospitalizations due to cardiovascular diseases from 2014 to 2018.

- EV_VCP.csv: Dataframe with the extreme events computed. Viracopos data was used for this.

## Analysis

This report presents an analysis of the the _incidence rate ratio_ for the [extreme climate events](https://github.com/climate-and-health-datasci-Unicamp/project-climatic-variations-cardiovascular-diseases/blob/main/notebooks/TR_2020_05_Extreme_climatic_events_for_Campinas.ipynb) using a case crossover design.

##**Case crossover**

In a case-crossover study design, each person serves as his own control. The period immediately before the adverse outcome (death or hospitalization) is then compared with a period when no adverse outcome occurred [Gordis].

As the analyzed events are rare, we used a random number to generate the control for each patient to avoid bias. The number used was between 30 and 300. We considered a period of exposure of 5 days, i.e., we compared the exposure to extreme climatic events on the five days before the (case) with the exposure five days before the control.

Then, we performed a conditional logistic regression to obtain estimates of odds ratios (OR) and theirs 95 \% confidence interval. Because of the sampling strategy and the small initial risk (chance of a cardiovascular outcome in the city of Campinas), these ORs can be considered a reasonable estimate of the incidence rate ratio (IRR) [Davies].

Besides the extreme event of interest, meteorological parameters of maximum temperature, average pressure, and maximum humidity were included in all the regressions. Minimum temperature and humidity were not included due to the high correlation with the maximum temperature and humidity.

Because we used a random number to define the control, the results will be different each time. We used the mean value of 20 regressions to estimate the RR associated with each extreme event.

##**Import libraries**

In [1]:
#import python libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import math
import statistics as stat
import scipy.stats as stats
import datetime as dt
import random

#plots
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
import seaborn as sns
import matplotlib.ticker as mticker

#array 
from array import array
from itertools import repeat

#logistic regression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

#files
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


##**Load data**

###**Climatic data**

In [2]:
#-------------------------------------------------------------------#
#                          Load Viracopos data                      #
#-------------------------------------------------------------------#

#Load humidity dataframe
df_VCP = pd.read_csv('EV_VCP.csv')
df_VCP = df_VCP.drop(columns = {'Unnamed: 0'})
df_VCP['DATE'] = pd.to_datetime(df_VCP['DATE'])

In [3]:
#-------------------------------------------------------------------#
#                             Null rows                             #
#-------------------------------------------------------------------#
print("Percentage of null values VCP (2001-2018) \n")

print("TMIN:", round((len(df_VCP[df_VCP['TMIN'].isnull()]))/len(df_VCP)*100,2),"%")
print("TMAX:", round((len(df_VCP[df_VCP['TMAX'].isnull()]))/len(df_VCP)*100,2),"%")
print("AVGPRESSURE:", round((len(df_VCP[df_VCP['AVGPRESSURE'].isnull()]))/len(df_VCP)*100,2),"%")
print("HMIN:", round((len(df_VCP[df_VCP['HMIN'].isnull()]))/len(df_VCP)*100,2),"%")
print("HMAX:", round((len(df_VCP[df_VCP['HMAX'].isnull()]))/len(df_VCP)*100,2),"%")

print("\n")

print("Number of null rows \n")

print("TMIN:", len(df_VCP[df_VCP['TMIN'].isnull()]))
print("TMAX:", len(df_VCP[df_VCP['TMAX'].isnull()]))
print("AVGPRESSURE:",len(df_VCP[df_VCP['AVGPRESSURE'].isnull()]))
print("HMIN:", len(df_VCP[df_VCP['HMIN'].isnull()]))
print("HMAX:", len(df_VCP[df_VCP['HMAX'].isnull()]))

Percentage of null values VCP (2001-2018) 

TMIN: 0.18 %
TMAX: 0.18 %
AVGPRESSURE: 0.02 %
HMIN: 0.05 %
HMAX: 0.05 %


Number of null rows 

TMIN: 12
TMAX: 12
AVGPRESSURE: 1
HMIN: 3
HMAX: 3


In [4]:
#fill na with mean values
df_VCP['HMIN'].fillna(df_VCP['HMIN'].mean(), inplace=True)
df_VCP['HMAX'].fillna(df_VCP['HMAX'].mean(), inplace=True)
df_VCP['TMIN'].fillna(df_VCP['TMIN'].mean(), inplace=True)
df_VCP['TMAX'].fillna(df_VCP['TMAX'].mean(), inplace=True)
df_VCP['AVGPRESSURE'].fillna(df_VCP['AVGPRESSURE'].mean(), inplace=True)

###**Health data**

In [6]:
#-------------------------------------------------------------------#
#                 Circulatory hospitalizations                      #
#-------------------------------------------------------------------#

df_hosp = pd.read_csv('hospitalizações_circulatório.csv')
df_hosp = df_hosp.drop(columns = {'Unnamed: 0','Hora','Número Paciente','Descrição CID'}) #drop unneeded columns
df_hosp = df_hosp.rename(columns = {'Data':'DATE','Idade': 'IDADE','Sexo': 'SEXO'}) #rename Data to DATE to merge dataframes
df_hosp = df_hosp[(df_hosp.DATE !='2012-02-29')&(df_hosp.DATE !='2016-02-29')] #remove leap year dates (02-29)
df_hosp = df_hosp.sort_values('DATE')

##**Functions**

In [7]:
def case_crossover_random_exposed(df_health,df_climate,event,show):

  #convert to datetime
  df_health['DATE'] = pd.to_datetime(df_health['DATE'])
  df_climate['DATE'] = pd.to_datetime(df_climate['DATE'])

  #exposition: 5 days before extreme event
  df_climate['exposed'] = df_climate[event].rolling(min_periods=1, window=5).sum()
  df_climate['exposed']= np.where(df_climate['exposed']==0,0,1)

  #cases 
  df_health1 = df_health.copy() #make a copy of health database
  df_health1['case'] = 1 #all the deaths are considered cases
  cases = pd.merge(df_health1, df_climate, on = 'DATE', how='outer') #merge with climate data
  
  #control - use a random number of days to determine the control
  df_health2 = df_health.copy() #make a copy of health database
  random_list = [] #create a random list with the size of the dataframe
  for i in range(0,len(df_health2)):
    random_list.append(random.randint(30, 300))
  #use a auxiliar column to shift dates
  df_health2['aux'] = random_list 
  df_health2['aux'] = df_health2['aux'].apply(lambda x: timedelta(days=x))
  df_health2['DATE'] = df_health2['DATE'] - df_health2['aux'] #shift dates
  df_health2['case']= 0 #all the shift dates are controls (a death didn't occured)
  control = pd.merge(df_health2, df_climate, on = 'DATE', how='outer') #merge with climate data

  #concat both dataframes
  df = pd.concat([cases, control])
  df = df[~df['case'].isnull()] #drop days without health data
  df = df[~df['exposed'].isnull()] #drop days without climate data
  df = df.sort_values('DATE') #sort by dates

  #cases and controls, exposed and unexposed
  case_exposed = len(df[(df['case']==1) & (df['exposed']==1)])
  case_unexposed = len(df[(df['case']==1) & (df['exposed']==0)])
  control_exposed = len(df[(df['case']==0) & (df['exposed']==1)])
  control_unexposed = len(df[(df['case']==0) & (df['exposed']==0)])
  
  #if show=True, print
  if show:
    print(f"case exposed: {case_exposed} \ncase unexposed: {case_unexposed} \ncontrol exposed: {control_exposed} \ncontrol unexposed: {control_unexposed} \n")

  #logistic regression - only with the event (exposition) as variable
  y = df['case']
  x = df['exposed']
  x = sm.add_constant(x)
  model = sm.Logit(y, x)
  result = model.fit(method='newton',disp=False)
  IRR = np.exp(result.params[1])
  
  #if show=True, print
  if show:
    print("IRR:", round(np.exp(result.params[1]),2))

  #return dataframe
  return df

In [8]:
def regression(data,event,variables):
        ###data - health data
        ###event - extreme event we want to compute the RR
        ###variables - explanatory variables

  #get the dataframe with cases and controls
  df = case_crossover_random_exposed(data, df_VCP,event,False)
  #y is the result - case or control
  y = df['case']
  #x are all of the explanotory variables
  x = df[variables]

  #perfom a logistic regression regression
  model = sm.Logit(y, x)
  result = model.fit(method='newton', disp=False)
  #rate ratio is the exponential of the coeficient associated to the extreme event
  RR = round(np.exp(result.params[0]),2)
  #confidence intervals
  ci = result.conf_int()
  lower_ci = round(np.exp(ci.iloc[0,0]),2)
  upper_ci = round(np.exp(ci.iloc[0,1]),2)
  
  #return the RR and CI
  return RR, lower_ci, upper_ci

In [9]:
def mean_rr(number,data,event,variables):
  ### number - times of regressions to compute the mean
  ### data - health database
  ###event - extreme event we want to compute the RR
  ###variables - explanatory variables

  #insert the exposition to the extreme event in the variables list
  variables.insert(0,'exposed')
  
  #create lists to store results
  irr_list = []
  lower_ci_list = []
  upper_ci_list = []

  #perform the regression n times
  for i in range(0, number+1):
    irr, lower_ci, upper_ci = regression(data,event,variables)
    irr_list.append(irr)
    lower_ci_list.append(lower_ci)
    upper_ci_list.append(upper_ci)

  #get the mean values
  print("IRR:",irr_list)
  print("IRR:", round(stat.mean(irr_list),2))
  print("lower ci:",lower_ci_list)
  print("upper ci:",upper_ci_list)
  print(f"CI: {round(stat.mean(lower_ci_list),2)} - {round(stat.mean(upper_ci_list),2)}")

##**Analysis**

Perform the regression 20 times and take the mean values

In [10]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_hosp,'above_temp_range',columns)

IRR: [0.86, 0.84, 0.88, 0.88, 0.9, 0.87, 0.9, 0.88, 0.88, 0.87, 0.89, 0.86, 0.88, 0.86, 0.89, 0.91, 0.92, 0.9, 0.9, 0.86, 0.9]
IRR: 0.88
lower ci: [0.81, 0.8, 0.83, 0.83, 0.85, 0.82, 0.85, 0.83, 0.83, 0.82, 0.83, 0.81, 0.83, 0.81, 0.84, 0.86, 0.87, 0.85, 0.85, 0.81, 0.85]
upper ci: [0.91, 0.9, 0.93, 0.93, 0.96, 0.93, 0.96, 0.94, 0.93, 0.93, 0.94, 0.92, 0.94, 0.91, 0.95, 0.97, 0.98, 0.96, 0.95, 0.92, 0.96]
CI: 0.83 - 0.94


In [11]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_hosp,'above_temp_dif',columns)

IRR: [1.07, 1.02, 1.07, 1.06, 1.07, 1.09, 1.08, 1.06, 1.04, 1.1, 1.21, 1.05, 1.09, 1.07, 1.01, 1.12, 1.11, 1.07, 1.05, 1.12, 1.1]
IRR: 1.08
lower ci: [0.96, 0.92, 0.96, 0.95, 0.96, 0.98, 0.97, 0.95, 0.93, 0.99, 1.08, 0.95, 0.98, 0.96, 0.91, 1.0, 1.0, 0.96, 0.94, 1.0, 0.99]
upper ci: [1.19, 1.13, 1.2, 1.18, 1.19, 1.21, 1.2, 1.18, 1.16, 1.23, 1.35, 1.17, 1.21, 1.19, 1.13, 1.25, 1.24, 1.19, 1.17, 1.24, 1.23]
CI: 0.97 - 1.2


In [12]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_hosp,'LPW',columns)

IRR: [1.13, 1.12, 1.12, 1.08, 1.22, 1.08, 1.18, 1.13, 1.11, 1.12, 1.16, 1.08, 1.08, 1.13, 1.1, 1.16, 1.08, 1.07, 1.2, 1.15, 1.14]
IRR: 1.13
lower ci: [1.01, 1.0, 1.0, 0.97, 1.09, 0.97, 1.05, 1.01, 0.99, 1.0, 1.04, 0.97, 0.97, 1.01, 0.98, 1.04, 0.97, 0.95, 1.07, 1.03, 1.02]
upper ci: [1.26, 1.25, 1.25, 1.21, 1.37, 1.21, 1.32, 1.27, 1.24, 1.26, 1.3, 1.21, 1.21, 1.26, 1.23, 1.3, 1.21, 1.19, 1.35, 1.29, 1.28]
CI: 1.01 - 1.26


In [13]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_hosp,'HPW',columns)

IRR: [1.03, 1.05, 1.06, 1.07, 1.01, 1.07, 1.02, 1.05, 1.05, 1.06, 1.03, 1.05, 1.01, 1.05, 1.0, 1.0, 1.03, 1.08, 1.05, 1.06, 1.02]
IRR: 1.04
lower ci: [0.97, 0.98, 0.99, 1.0, 0.95, 1.0, 0.96, 0.98, 0.99, 1.0, 0.97, 0.98, 0.95, 0.98, 0.94, 0.94, 0.97, 1.01, 0.98, 1.0, 0.96]
upper ci: [1.1, 1.12, 1.13, 1.14, 1.08, 1.14, 1.09, 1.12, 1.12, 1.14, 1.1, 1.12, 1.08, 1.12, 1.07, 1.07, 1.1, 1.15, 1.12, 1.14, 1.09]
CI: 0.98 - 1.11


In [14]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_hosp,'above_pressure_dif',columns)

IRR: [0.97, 0.96, 0.96, 1.02, 0.97, 0.97, 0.94, 0.96, 0.98, 0.97, 0.98, 0.99, 0.97, 0.99, 0.96, 0.98, 1.0, 0.98, 0.98, 0.96, 0.99]
IRR: 0.98
lower ci: [0.92, 0.91, 0.91, 0.96, 0.93, 0.92, 0.89, 0.91, 0.93, 0.92, 0.93, 0.94, 0.92, 0.94, 0.92, 0.93, 0.95, 0.93, 0.93, 0.91, 0.94]
upper ci: [1.02, 1.01, 1.01, 1.07, 1.03, 1.02, 0.99, 1.01, 1.03, 1.02, 1.04, 1.04, 1.02, 1.04, 1.02, 1.03, 1.05, 1.03, 1.03, 1.01, 1.04]
CI: 0.93 - 1.03


In [15]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_hosp,'LHW',columns)

IRR: [1.03, 1.05, 1.03, 1.02, 1.04, 1.02, 1.02, 1.05, 1.05, 1.05, 0.97, 1.05, 1.08, 1.04, 1.02, 1.02, 1.08, 0.96, 1.06, 1.03, 1.05]
IRR: 1.03
lower ci: [0.95, 0.97, 0.95, 0.94, 0.96, 0.94, 0.94, 0.97, 0.96, 0.96, 0.89, 0.97, 1.0, 0.96, 0.94, 0.94, 1.0, 0.89, 0.98, 0.95, 0.97]
upper ci: [1.12, 1.14, 1.11, 1.11, 1.13, 1.11, 1.11, 1.14, 1.13, 1.13, 1.05, 1.14, 1.17, 1.13, 1.1, 1.11, 1.17, 1.05, 1.15, 1.11, 1.14]
CI: 0.95 - 1.12


In [16]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_hosp,'HHW',columns)

IRR: [0.33, 0.35, 0.33, 0.36, 0.39, 0.4, 0.34, 0.44, 0.39, 0.4, 0.36, 0.37, 0.44, 0.37, 0.37, 0.35, 0.38, 0.38, 0.36, 0.33, 0.35]
IRR: 0.37
lower ci: [0.25, 0.27, 0.25, 0.27, 0.29, 0.3, 0.25, 0.33, 0.29, 0.3, 0.27, 0.28, 0.33, 0.28, 0.28, 0.26, 0.28, 0.29, 0.27, 0.25, 0.26]
upper ci: [0.44, 0.47, 0.44, 0.48, 0.52, 0.54, 0.45, 0.59, 0.52, 0.54, 0.48, 0.5, 0.59, 0.5, 0.5, 0.46, 0.5, 0.51, 0.47, 0.44, 0.46]
CI: 0.28 - 0.5


In [17]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_hosp,'above_humidity_range',columns)

IRR: [0.92, 0.9, 0.92, 0.94, 0.94, 0.96, 0.94, 0.93, 0.93, 0.93, 0.95, 0.92, 0.94, 0.93, 0.93, 0.93, 0.93, 0.95, 0.92, 0.91, 0.93]
IRR: 0.93
lower ci: [0.88, 0.86, 0.87, 0.89, 0.89, 0.91, 0.89, 0.89, 0.88, 0.88, 0.91, 0.87, 0.89, 0.89, 0.89, 0.88, 0.89, 0.91, 0.87, 0.87, 0.88]
upper ci: [0.97, 0.95, 0.97, 0.99, 0.99, 1.01, 0.99, 0.98, 0.98, 0.98, 1.0, 0.97, 0.99, 0.98, 0.98, 0.97, 0.98, 1.01, 0.97, 0.96, 0.98]
CI: 0.89 - 0.98


In [18]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_hosp,'above_humidity_dif',columns)

IRR: [1.01, 0.97, 1.05, 1.03, 1.07, 0.97, 1.04, 1.02, 1.1, 1.04, 1.02, 0.96, 1.08, 0.96, 1.03, 0.96, 0.98, 1.01, 1.04, 1.01, 1.05]
IRR: 1.02
lower ci: [0.91, 0.87, 0.94, 0.92, 0.96, 0.88, 0.94, 0.92, 0.99, 0.94, 0.92, 0.87, 0.97, 0.86, 0.93, 0.87, 0.89, 0.91, 0.94, 0.91, 0.95]
upper ci: [1.12, 1.07, 1.16, 1.14, 1.19, 1.08, 1.16, 1.13, 1.22, 1.16, 1.13, 1.07, 1.2, 1.06, 1.14, 1.07, 1.09, 1.12, 1.16, 1.12, 1.17]
CI: 0.92 - 1.13


##**References**

L. Gordis, “Epidemiology, Saunders Elsevier", Philadelphia, Pa, USA, 2013.

H. T. O. Davies, I. K. Crombie, and M. Tavakoli, “When can odds ratios mislead?”Bmj, vol.316, no. 7136, pp. 989–991, 1998.